In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
fabianavinci_guitar_chords_v3_path = kagglehub.dataset_download('fabianavinci/guitar-chords-v3')

print('Data source import complete.')

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import os

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# This code prints all input files filepaths
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of https://www.kaggle.com/code/olenkaaaaa/aipres/edit/run/207589001the current session

In [ ]:
!pip install librosa tensorflow

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models, Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, Dropout, BatchNormalization, LayerNormalization,
    MultiHeadAttention, Flatten, Embedding, GlobalAveragePooling1D,
    LSTM, GRU, Multiply, Softmax, Conv2D, MaxPooling2D, Activation
)
from tensorflow.keras.optimizers import Adam

In [ ]:
def extract_features(file_path, n_mfcc=40):
    audio, sample_rate = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=n_mfcc)
    mfccs_processed = np.mean(mfccs.T, axis=0)
    return mfccs_processed

In [ ]:
train_directory = '/root/.cache/kagglehub/datasets/fabianavinci/guitar-chords-v3/versions/1/Training'
test_directory = '/root/.cache/kagglehub/datasets/fabianavinci/guitar-chords-v3/versions/1/Test'

chord_map = {'Am': 0, 'Bb': 1, 'Bdim': 2, 'C': 3, 'Dm': 4, 'Em': 5, 'F': 6, 'G': 7}

def load_data_from_directory(directory, label_map):
    X = []
    y = []
    for chord, label in label_map.items():
        chord_path = os.path.join(directory, chord)
        if os.path.exists(chord_path):
            for file_name in os.listdir(chord_path):
                if file_name.endswith('.wav'):
                    file_path = os.path.join(chord_path, file_name)
                    try:
                        features = extract_features(file_path)
                        X.append(features)
                        y.append(label)
                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")
    return np.array(X), np.array(y)

In [ ]:
def process_single_file(file_path, chord_label, label_map):
    features = extract_features(file_path)
    if chord_label in label_map.keys():
        label = label_map[chord_label]
    else:
        label = -1
    return features, label

In [ ]:
X_train, y_train = load_data_from_directory(train_directory, chord_map)
X_test, y_test = load_data_from_directory(test_directory, chord_map)

In [ ]:
print("Training data shape:", X_train.shape, y_train.shape)
print("Testing data shape:", X_test.shape, y_test.shape)

In [ ]:
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(chord_map), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=50, validation_split=0.1, batch_size=32)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.2f}')

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()